<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/QAlpha_Knapsack_July.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install docplex
!pip install cplex

     |████████████████████████████████| 30.9 MB 32 kB/s 


In [5]:
from docplex.mp.model import *
from docplex.mp.utils import *
from docplex.util.status import JobSolveStatus
from docplex.mp.conflict_refiner import ConflictRefiner, VarUbConstraintWrapper, VarLbConstraintWrapper
from docplex.mp.relaxer import Relaxer
import time
import sys
import operator

import pandas as pd
import numpy as np
import math

import codecs
import sys

# Handle output of unicode strings
if sys.version_info[0] < 3:
    sys.stdout = codecs.getwriter('utf8')(sys.stdout)


from pandas.api.types import is_string_dtype


#Load all input data as a map { data_name: data_frame }
inputs = {'july': pd.read_csv('./july.csv')}
# This will hold all outputs as a map { data_name: data_frame }
outputs = {}

# we use a lock to access ``outputs``. This allows solves() to
# be aborted without race condition in data writting
import threading
output_lock = threading.Lock()


def helper_check_data_type(df, column, df_label, check_type):
    if not column in df.columns:
        print('Column "%s" does not exist in table "%s"' % (column, df_label))
        return False
    non_nan_values = df[column][~df[column].isnull()]
    if check_type == 'INTEGER':
        k = non_nan_values.dtype.kind
        if k != 'i':
            if k == 'f':
                non_integer_values = non_nan_values.values[np.where([not x.is_integer() for x in non_nan_values])]
                if len(non_integer_values) > 0:
                    print('Column "%s" of table "%s" contains non-integer value(s) which violates expected type: %s' % (column, df_label, non_integer_values))
                    return False
            else:
                print('Column "%s" of table "%s" is non-numeric which violates expected type: %s' % (column, df_label, non_nan_values.values))
                return False
    elif check_type == 'FLOAT' or check_type == 'NUMBER':
        non_float_values = non_nan_values.values[np.where([not isinstance(x, (int, float)) for x in non_nan_values])]
        k = non_nan_values.dtype.kind
        if not k in ['i', 'f']:
            print('Column "%s" of table "%s" contains non-float value(s) which violates expected type: %s' % (column, df_label, non_float_values))
            return False
    elif check_type == 'BOOLEAN':
        non_bool_values = non_nan_values.values[np.where([not isinstance(x, bool) for x in non_nan_values])]
        if len(non_bool_values) > 0:
            print('Column "%s" of table "%s" contains non-boolean value(s) which violates expected type: %s' % (column, df_label, non_bool_values))
            return False
    elif check_type == 'Date' or check_type == 'DateTime':
        try:
            pd.to_datetime(non_nan_values)
        except ValueError as e:
            print('Column "%s" of table "%s" cannot be converted to a DateTime : %s' % (column, df_label, str(e)))
            return False
    elif check_type == 'Time':
        try:
            pd.to_timedelta(non_nan_values)
        except ValueError as e:
            try:
                # Try appending ':00' in case seconds are not represented in time
                pd.to_timedelta(non_nan_values + ':00')
            except ValueError as e:
                print('Column "%s" of table "%s" cannot be converted to a Time : %s' % (column, df_label, str(e)))
                return False
    elif check_type == 'STRING':
        if not is_string_dtype(non_nan_values):
            print('Column "%s" of table "%s" is not of type "String"' % (column, df_label))
            return False
    else:
        raise Exception('Invalid check_type: %s' % check_type)
    return True


def helper_check_foreignKey_values(source_df, source_column, source_df_label, target_df, target_column, target_df_label):
    non_nan_values = source_df[source_column][~source_df[source_column].isnull()]
    invalid_FK_values = non_nan_values[~non_nan_values.isin(target_df[target_column])].values
    if len(invalid_FK_values) > 0:
        print('FK Column "%s" of table "%s" contains values that do not exist in PK column "%s" of target table "%s": %s' % (source_column, source_df_label, target_column, target_df_label, invalid_FK_values))
        return False
    return True


def helper_check_unique_primaryKey_values(df, key_cols, df_label):
    df_grp = df.groupby(key_cols).size()
    invalid_pk_values = df_grp[df_grp > 1].reset_index()[key_cols].values
    if len(invalid_pk_values) > 0:
        print('Non-unique values for PK of table "%s": %s' % (df_label, invalid_pk_values))
        return False
    return True


# Label constraint
def helper_add_labeled_cplex_constraint(mdl, expr, label, context=None, columns=None):
    global expr_counter
    if isinstance(expr, np.bool_):
        expr = expr.item()
    if isinstance(expr, bool):
        pass  # Adding a trivial constraint: if infeasible, docplex will raise an exception it is added to the model
    else:
        expr.name = '_L_EXPR_' + str(len(expr_to_info) + 1)
        if columns:
            ctxt = ", ".join(str(getattr(context, col)) for col in columns)
        else:
            if context:
                ctxt = context.Index if isinstance(context.Index, str) is not None else ", ".join(context.Index)
            else:
                ctxt = None
        expr_to_info[expr.name] = (label, ctxt)
    mdl.add(expr)

def helper_get_column_name_for_property(property):
    return helper_property_id_to_column_names_map.get(property, 'unknown')


def helper_get_index_names_for_type(dataframe, type):
    if not is_pandas_dataframe(dataframe):
        return None
    return [name for name in dataframe.index.names if name in helper_concept_id_to_index_names_map.get(type, [])]


helper_concept_id_to_index_names_map = {
    'cItem': ['id_of_July'],
    'july': ['id_of_July']}
helper_property_id_to_column_names_map = {
    'july.city': 'city',
    'july.sick': 'sick',
    'july.gdp': 'gdp'}


# Data model definition for each table
# Data collection: list_of_July ['sick', 'gdp', 'city']

# Create a pandas Dataframe for each data table
list_of_July = inputs[u'july']
list_of_July = list_of_July[[u'sick', u'gdp', u'city']].copy()
list_of_July.rename(columns={u'sick': 'sick', u'gdp': 'gdp', u'city': 'city'}, inplace=True)

# Perform input data checking against schema configured in Modelling Assistant along with unicity of PK values
data_check_result = True
# --- Handling data checking for table: july
data_check_result &= helper_check_data_type(list_of_July, 'sick', 'july', 'NUMBER')
data_check_result &= helper_check_data_type(list_of_July, 'gdp', 'july', 'NUMBER')
data_check_result &= helper_check_unique_primaryKey_values(list_of_July, ['city'], 'july')
if not data_check_result:
    # Stop execution here
    raise Exception('Data checking detected errors')

# Force column type for non-numeric columns
list_of_July = list_of_July.astype({'city': object})

# Set index when a primary key is defined
list_of_July.set_index('city', inplace=True)
list_of_July.sort_index(inplace=True)
list_of_July.index.name = 'id_of_July'


#VC: extra scalars unable to be defined by IBM's auto AI
gdp_remain_ratio = 0.2
sick_remain_ratio = 0.1



def build_model():
    mdl = Model()

    # Definition of model variables
    list_of_July['selectionVar'] = mdl.binary_var_list(len(list_of_July))


    # Definition of model
    # Objective cMaximizeGoalSelect#1-
    # Combine weighted criteria: 
    # 	cMaximizeGoalSelect cMaximizeGoalSelect 1.2{
    # 	numericExpr = cSelection__july__ / july / gdp,
    # 	scaleFactorExpr = 1,
    # 	(static) goalFilter = null} with weight 8.0
    # 	cMinimizeGoalSelect cMinimizeGoalSelect 1.2{
    # 	numericExpr = cSelection__july__ / july / sick,
    # 	scaleFactorExpr = 1,
    # 	(static) goalFilter = null} with weight 8.0
    # VC: 1 - selectionVar for modulo 2 effect
    list_of_July['conditioned_gdp'] = (list_of_July.selectionVar + gdp_remain_ratio * (1 - list_of_July.selectionVar)) * list_of_July.gdp
    agg_July_conditioned_gdp_SG1 = mdl.sum(list_of_July.conditioned_gdp)
    list_of_July['conditioned_sick'] = (list_of_July.selectionVar + sick_remain_ratio * (1 - list_of_July.selectionVar)) * list_of_July.sick
    agg_July_conditioned_sick_SG2 = mdl.sum(list_of_July.conditioned_sick)
    
    kpis_expression_list = [
        (-1, 128.0, agg_July_conditioned_gdp_SG1, 1, 0, u'total gdp of julies over all selections'),
        (1, 128.0, agg_July_conditioned_sick_SG2, 1, 0, u'total sick of julies over all selections')]
    custom_code.update_goals_list(kpis_expression_list)
    
    for _, kpi_weight, kpi_expr, kpi_factor, kpi_offset, kpi_name in kpis_expression_list:
        mdl.add_kpi(kpi_expr, publish_name=kpi_name)
    
    mdl.minimize(sum([kpi_sign * kpi_weight * ((kpi_expr * kpi_factor) - kpi_offset) for kpi_sign, kpi_weight, kpi_expr, kpi_factor, kpi_offset, kpi_name in kpis_expression_list]))
    
    # [ST_1] Constraint : cGlobalRelationalConstraint#1_cGlobalRelationalConstraint
    # total sick of julies over all selections is less than or equal to 500000
    # Label: CT_1_total_sick_of_julies_over_all_selections_is_less_than_or_equal_to_500000
    list_of_July['conditioned_sick'] = list_of_July.selectionVar * list_of_July.sick
    agg_July_conditioned_sick_lhs = mdl.sum(list_of_July.conditioned_sick)
    helper_add_labeled_cplex_constraint(mdl, agg_July_conditioned_sick_lhs <= 500000, u'total sick of julies over all selections is less than or equal to 500000')


    return mdl


def solve_model(mdl):
    mdl.parameters.timelimit = 120
    # Call to custom code to update parameters value
    custom_code.update_solver_params(mdl.parameters)
    # Update parameters value according to environment variables definition
    cplex_param_env_prefix = 'ma.cplex.'
    cplex_params = [name.qualified_name for name in mdl.parameters.generate_params()]
    for param in cplex_params:
        env_param = cplex_param_env_prefix + param
        param_value = get_environment().get_parameter(env_param)
        if param_value:
            # Updating parameter value
            print("Updated value for parameter %s = %s" % (param, param_value))
            parameters = mdl.parameters
            for p in param.split('.')[1:]:
                parameters = parameters.__getattribute__(p)
            parameters.set(param_value)

    msol = mdl.solve(log_output=True)
    if not msol:
        print("!!! Solve of the model fails")
        if mdl.get_solve_status() == JobSolveStatus.INFEASIBLE_SOLUTION or mdl.get_solve_status() == JobSolveStatus.INFEASIBLE_OR_UNBOUNDED_SOLUTION:
            crefiner = ConflictRefiner()
            conflicts = crefiner.refine_conflict(model, log_output=True)
            export_conflicts(conflicts)
            
    print('Solve status: %s' % mdl.get_solve_status())
    if mdl.get_solve_status() == JobSolveStatus.UNKNOWN:
        print('UNKNOWN cause: %s' % mdl.get_solve_details().status)
    mdl.report()
    return msol


expr_to_info = {}


def export_conflicts(conflicts):
    # Display conflicts in console
    print('Conflict set:')
    list_of_conflicts = pd.DataFrame(columns=['constraint', 'context', 'detail'])
    for conflict, index in zip(conflicts, range(len(conflicts))):
        st = conflict.status
        ct = conflict.element
        label, context = expr_to_info.get(conflict.name, ('N/A', conflict.name))
        label_type = type(conflict.element)
        if isinstance(conflict.element, VarLbConstraintWrapper) \
                or isinstance(conflict.element, VarUbConstraintWrapper):
            label = 'Upper/lower bound conflict for variable: {}'.format(conflict.element._var)
            context = 'Decision variable definition'
            ct = conflict.element.get_constraint()

        # Print conflict information in console
        print("Conflict involving constraint: %s, \tfor: %s -> %s" % (label, context, ct))
        list_of_conflicts = list_of_conflicts.append({'constraint': label, 'context': str(context), 'detail': ct},
                                                     ignore_index=True)

    # Update of the ``outputs`` dict must take the 'Lock' to make this action atomic,
    # in case the job is aborted
    global output_lock
    with output_lock:
        outputs['list_of_conflicts'] = list_of_conflicts


def export_solution(msol):
    start_time = time.time()
    mdl = msol.model
    list_of_July_solution = pd.DataFrame(index=list_of_July.index)
    list_of_July_solution['selectionVar'] = msol.get_values(list_of_July.selectionVar.values)
    NotSelectedJulies = pd.DataFrame(index=list_of_July.index)
    
    # Adding extra columns based on Solution Schema
    NotSelectedJulies['july sick'] = list_of_July['sick']
    NotSelectedJulies['july gdp'] = list_of_July['gdp']
    
    NotSelectedJulies['_INTERNAL_selectionVar'] = list_of_July_solution.selectionVar
    NotSelectedJulies = NotSelectedJulies[NotSelectedJulies._INTERNAL_selectionVar <= 0.5]
    NotSelectedJulies = NotSelectedJulies.drop('_INTERNAL_selectionVar', axis='columns')

    SelectedJulies = pd.DataFrame(index=list_of_July.index)
    SelectedJulies['selected julies decision'] = list_of_July_solution['selectionVar']
    SelectedJulies['july sick'] = list_of_July['sick']
    SelectedJulies['july gdp'] = list_of_July['gdp']
    
    SelectedJulies['_INTERNAL_selectionVar'] = list_of_July_solution.selectionVar
    SelectedJulies = SelectedJulies[SelectedJulies._INTERNAL_selectionVar > 0.5]
    SelectedJulies = SelectedJulies.drop('_INTERNAL_selectionVar', axis='columns')


    # Update of the ``outputs`` dict must take the 'Lock' to make this action atomic,
    # in case the job is aborted
    global output_lock
    with output_lock:
        outputs['NotSelectedJulies'] = NotSelectedJulies[['july sick', 'july gdp']].reset_index().rename(columns= {'id_of_July': 'july'})
        outputs['SelectedJulies'] = SelectedJulies[['selected julies decision', 'july sick', 'july gdp']].reset_index().rename(columns= {'id_of_July': 'july'})
        custom_code.post_process_solution(msol, outputs)

    elapsed_time = time.time() - start_time
    print('solution export done in ' + str(elapsed_time) + ' secs')
    return


# Import custom code definition if module exists
try:
    from custom_code import CustomCode
    custom_code = CustomCode(globals())
except ImportError:
    # Create a dummy anonymous object for custom_code
    custom_code = type('', (object,), {'preprocess': (lambda *args: None),
                                       'update_goals_list': (lambda *args: None),
                                       'update_model': (lambda *args: None),
                                       'update_solver_params': (lambda *args: None),
                                       'post_process_solution': (lambda *args: None)})()


from docplex.mp.progress import ProgressListener
from docplex.util.environment import add_abort_callback, remove_abort_callback
from functools import partial


class SolutionListener(ProgressListener):
    def __init__(self):
        super(SolutionListener, self).__init__()
        self._cpx_incumbent_sol = None

    def requires_solution(self):
        return True

    def notify_solution(self, cpx_incumbent_sol):
        self._cpx_incumbent_sol = cpx_incumbent_sol


def save_and_write_last_solution_callback(sol_listener, outputs):
    if sol_listener._cpx_incumbent_sol:
        export_solution(sol_listener._cpx_incumbent_sol)
    write_all_outputs(outputs)


solution_listener = SolutionListener()

# Custom pre-process
custom_code.preprocess()

print('* building wado model')
start_time = time.time()
model = build_model()

# Model customization
custom_code.update_model(model)

#
model.add_progress_listener(solution_listener)
env = get_environment()
# Remove default abort callbacks
for cb in env.abort_callbacks:
    remove_abort_callback(cb)
# Add new abort callback to store latest found solution, if any
add_abort_callback(partial(save_and_write_last_solution_callback, solution_listener, outputs))

elapsed_time = time.time() - start_time
print('model building done in ' + str(elapsed_time) + ' secs')

print('* running wado model')
start_time = time.time()
msol = solve_model(model)
elapsed_time = time.time() - start_time
print('model solve done in ' + str(elapsed_time) + ' secs')
if msol:
    export_solution(msol)

* building wado model
model building done in 0.028308391571044922 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Legacy callback                                  i
Found incumbent of value -3.2541974e+08 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 1 rows, 50 columns, and 50 nonzeros.
Reduced MIP has 50 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.07 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 1 rows, 50 columns, and 50 nonzeros.
Reduced MIP has 50 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.07 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 2 threads.
Root relaxation solution time = 0.00 sec. (0.02 

In [6]:
print(outputs['SelectedJulies'])

              july  selected julies decision  july sick  july gdp
0          Atlanta                       1.0      46902    397261
1        Baltimore                       1.0       7030    205313
2           Boston                       1.0       4824    463570
3        Charlotte                       1.0      12387    169862
4          Chicago                       1.0      19759    689464
5       Cincinnati                       1.0       6020    141042
6        Cleveland                       1.0       5552    134369
7         Columbus                       1.0       5752    129328
8           Dallas                       1.0      60840    512509
9           Denver                       1.0       5336    214157
10         Detroit                       1.0       6543    267731
11        Hartford                       1.0        782     99465
12        Honolulu                       1.0        414     69252
13         Houston                       1.0      56767    478778
14    Indi